# Entraînement du Modèle Morningstar sur Google Colab

Ce notebook guide à travers les étapes nécessaires pour entraîner le modèle hybride multi-tâches Morningstar dans l'environnement Google Colab.

**Étapes principales :**
1. Configuration de l'environnement (clonage du repo, installation des dépendances).
2. Chargement et préparation des données pour un actif spécifique.
3. Définition et compilation de l'architecture du modèle Morningstar.
4. Entraînement du modèle avec suivi des performances.
5. Évaluation du modèle entraîné.
6. Visualisation des courbes d'apprentissage.

## 1. Configuration de l'Environnement

Nous devons d'abord cloner le dépôt contenant le code du projet et installer les dépendances nécessaires.

In [ ]:
# Cloner le dépôt (remplacez par l'URL de votre dépôt)
# Assurez-vous que le dépôt est accessible (public ou via token/clé SSH si privé)
!git clone https://github.com/VOTRE_USER/VOTRE_REPO.git CryptoRobot

# Se déplacer dans le répertoire du projet
%cd CryptoRobot/Morningstar

# Installer les dépendances
# Assurez-vous que requirements.txt est à jour
!pip install -r requirements.txt

**Note :** Si vos données ne sont pas dans le dépôt, vous devrez les rendre accessibles à Colab, par exemple en :
*   Montant votre Google Drive : `from google.colab import drive; drive.mount('/content/drive')` et en ajustant les chemins.
*   Téléchargeant les fichiers Parquet directement dans l'environnement Colab (ex: via `wget` ou l'interface Colab).

## 2. Chargement et Préparation des Données

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sys

# Ajouter le répertoire racine du projet au PYTHONPATH pour les imports locaux
# Ajustez si la structure clonée est différente
sys.path.append('/content/CryptoRobot/Morningstar') 

from model.training.data_loader import load_and_split_data

# --- Configuration --- 
ASSET_NAME = 'btc' # Choisir l'actif (btc, eth, sol, etc.)
# Ajuster le chemin si les données ne sont pas dans le repo cloné
DATA_DIR = Path('data/processed') 
FILE_PATH = DATA_DIR / f"{ASSET_NAME}_final.parquet"
LABEL_COLUMNS = ['trading_signal', 'volatility', 'market_regime']
VALIDATION_SPLIT = 0.2 # % des données pour la validation (fin de série)

print(f"Chargement des données pour : {ASSET_NAME} depuis {FILE_PATH}")

# Charger les données en tant que Tensors
try:
    X, y_dict = load_and_split_data(FILE_PATH, label_columns=LABEL_COLUMNS, as_tensor=True)
    print(f"Données chargées : X shape={X.shape}, Labels={list(y_dict.keys())}")
except FileNotFoundError:
    print(f"ERREUR : Le fichier {FILE_PATH} n'a pas été trouvé. Vérifiez le chemin et la disponibilité des données.")
except ValueError as e:
    print(f"ERREUR lors du chargement/split : {e}")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

# Séparation Train/Validation (temporelle)
num_samples = X.shape[0]
num_val_samples = int(num_samples * VALIDATION_SPLIT)
num_train_samples = num_samples - num_val_samples

X_train, X_val = X[:num_train_samples], X[num_train_samples:]
y_train_dict = {name: tensor[:num_train_samples] for name, tensor in y_dict.items()}
y_val_dict = {name: tensor[num_train_samples:] for name, tensor in y_dict.items()}

print(f"Séparation Train/Validation : Train={num_train_samples}, Val={num_val_samples}")
print(f"Shapes : X_train={X_train.shape}, X_val={X_val.shape}")
print(f"Labels Train : {[f'{k}:{v.shape}' for k, v in y_train_dict.items()]}")
print(f"Labels Val : {[f'{k}:{v.shape}' for k, v in y_val_dict.items()]}")

## 3. Définition et Compilation du Modèle Morningstar

In [ ]:
from model.architecture.enhanced_hybrid_model import build_enhanced_hybrid_model

# --- Configuration du Modèle --- 
INPUT_SHAPE = (X_train.shape[1],) 
# Doit correspondre à la cardinalité des labels de classification
NUM_TRADING_CLASSES = 5 # Ex: Strong Sell -> Strong Buy
NUM_REGIME_CLASSES = 3  # Ex: Bull, Bear, Sideways
LEARNING_RATE = 0.001

print("Construction du modèle Morningstar...")
model = build_enhanced_hybrid_model(input_shape=INPUT_SHAPE, 
                                    num_trading_classes=NUM_TRADING_CLASSES, 
                                    num_regime_classes=NUM_REGIME_CLASSES)

model.summary() # Afficher l'architecture

# Définir les pertes et métriques (doivent correspondre aux noms des couches de sortie)
losses = {
    'trading_signal_output': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    'volatility_output': tf.keras.losses.MeanSquaredError(),
    'market_regime_output': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
}
metrics = {
    'trading_signal_output': ['accuracy'],
    'volatility_output': [tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mae'],
    'market_regime_output': ['accuracy']
}
# loss_weights = {'trading_signal_output': 1.0, 'volatility_output': 0.5, 'market_regime_output': 0.8} # Optionnel

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

print("Compilation du modèle...")
model.compile(optimizer=optimizer, 
              loss=losses, 
              metrics=metrics)
              # loss_weights=loss_weights)

print("Modèle compilé.")

## 4. Entraînement du Modèle

In [ ]:
# --- Configuration de l'Entraînement ---
EPOCHS = 50
BATCH_SIZE = 32
MODEL_SAVE_DIR = Path('model/training') # Chemin relatif dans le projet cloné
MODEL_SAVE_PATH = MODEL_SAVE_DIR / f'{ASSET_NAME}_morningstar_colab.h5'

# Créer le répertoire de sauvegarde si nécessaire
MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True)

# Callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_SAVE_PATH,
    save_weights_only=False,
    monitor='val_loss', # Surveiller la perte totale de validation
    mode='min',
    save_best_only=True,
    verbose=1
)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, # Nb epochs sans amélioration avant arrêt
    verbose=1,
    restore_best_weights=True
)

print(f"Début de l'entraînement pour {EPOCHS} epochs...")
print(f"Le meilleur modèle sera sauvegardé dans : {MODEL_SAVE_PATH}")

history = model.fit(
    X_train,
    y_train_dict,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val_dict),
    callbacks=[checkpoint_callback, early_stopping_callback],
    verbose=1
)

print("Entraînement terminé.")

## 5. Évaluation du Modèle

Évaluons les performances du modèle (avec les poids restaurés du meilleur epoch grâce à `restore_best_weights=True` dans EarlyStopping) sur l'ensemble de validation.

In [ ]:
print("Évaluation du meilleur modèle sur l'ensemble de validation...")
results = model.evaluate(X_val, y_val_dict, batch_size=BATCH_SIZE, verbose=0)

print("Résultats de l'évaluation:")
results_dict = {}
try:
    for name, value in zip(model.metrics_names, results):
        results_dict[name] = value
        print(f"  - {name}: {value:.4f}")
except AttributeError:
    print("Impossible de récupérer model.metrics_names, affichage brut:", results)

# Optionnel : Charger explicitement le meilleur modèle sauvegardé si EarlyStopping n'a pas restauré les poids
# print(f"\nChargement du meilleur modèle depuis {MODEL_SAVE_PATH} pour vérification...")
# best_model = tf.keras.models.load_model(MODEL_SAVE_PATH)
# results_best = best_model.evaluate(X_val, y_val_dict, batch_size=BATCH_SIZE, verbose=0)
# print("Résultats du modèle chargé:")
# try:
#     for name, value in zip(best_model.metrics_names, results_best):
#         print(f"  - {name}: {value:.4f}")
# except AttributeError:
#     print("Impossible de récupérer model.metrics_names, affichage brut:", results_best)

## 6. Visualisation des Courbes d'Apprentissage

Visualisons l'évolution des pertes et des métriques clés pendant l'entraînement.

In [ ]:
history_dict = history.history

# Clés disponibles dans l'historique
print("\nClés disponibles dans l'historique:", history_dict.keys())

# Créer les graphiques
epochs_range = range(1, len(history_dict['loss']) + 1)

plt.figure(figsize=(15, 10))

# 1. Perte Totale
plt.subplot(2, 3, 1)
plt.plot(epochs_range, history_dict['loss'], label='Train Loss')
plt.plot(epochs_range, history_dict['val_loss'], label='Validation Loss')
plt.title('Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 2. Trading Signal Accuracy
plt.subplot(2, 3, 2)
plt.plot(epochs_range, history_dict['trading_signal_output_accuracy'], label='Train Accuracy')
plt.plot(epochs_range, history_dict['val_trading_signal_output_accuracy'], label='Validation Accuracy')
plt.title('Trading Signal Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 3. Volatility RMSE
plt.subplot(2, 3, 3)
plt.plot(epochs_range, history_dict['volatility_output_rmse'], label='Train RMSE')
plt.plot(epochs_range, history_dict['val_volatility_output_rmse'], label='Validation RMSE')
plt.title('Volatility RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.legend()

# 4. Market Regime Accuracy
plt.subplot(2, 3, 4)
plt.plot(epochs_range, history_dict['market_regime_output_accuracy'], label='Train Accuracy')
plt.plot(epochs_range, history_dict['val_market_regime_output_accuracy'], label='Validation Accuracy')
plt.title('Market Regime Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 5. Volatility MAE (si intéressante)
plt.subplot(2, 3, 5)
plt.plot(epochs_range, history_dict['volatility_output_mae'], label='Train MAE')
plt.plot(epochs_range, history_dict['val_volatility_output_mae'], label='Validation MAE')
plt.title('Volatility MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

## Fin du Notebook

Le modèle a été entraîné, évalué, et le meilleur modèle a été sauvegardé. Les courbes d'apprentissage donnent un aperçu de la convergence et des éventuels problèmes (sur-apprentissage, etc.).